# Imports

In [ ]:
from pathlib import Path
from asapdiscovery.data.backend.openeye import oechem, oespruce, load_openeye_pdb, load_openeye_sdfs
from asapdiscovery.modeling.modeling import get_oe_prep_opts, spruce_protein, du_to_complex
from asapdiscovery.data.schema.target import PreppedTarget
from asapdiscovery.data.schema.complex import PreppedComplex

In [ ]:
data_path = Path("/Users/alexpayne/Scientific_Projects/fragment-fold/data")

# load structure

In [ ]:
ref = load_openeye_pdb(data_path / "ns2-ns3b-zika-fragment-screen/fragment-screen-inputs/zikv_ns2b_ns3pro_apo.pdb")
success, spruce_error_message, spruced = spruce_protein(ref)

In [ ]:
success

In [ ]:
opts = get_oe_prep_opts()

In [ ]:
metadata = oespruce.OEStructureMetadata()

## get site residue

### figured out that the correct syntax should be, i.e.: 
"TYR:161: :B:0: "


The error returned by this command is: 
```
Warning: No BioAssembly transforms found, using input molecule as biounit: 
Warning: Iridium - Structure:  has no REMARK data
Processing BU # 1 with title: , chains AB
Warning: Failed to find a pocket associated with the input predicate
Warning: Failed, no pocket found with site residue predicate
Warning: Unable to parse and utilize specified site residue TYR:161: :B:0:  to define site in  - skipping
```


In [ ]:
dus = list(oespruce.OEMakeDesignUnits(spruced, metadata, opts, "TYR:161: :B:0: "))

Whereas the error returned by this command is: 
```
Warning: No BioAssembly transforms found, using input molecule as biounit: 
Warning: Iridium - Structure:  has no REMARK data
Processing BU # 1 with title: , chains AB
Warning: Failed, input predicate does not match any atom in molecule _(AB)
Warning: Failed, no pocket found with site residue predicate
Warning: Unable to parse and utilize specified site residue TYR:161: :A to define site in  - skipping
```

In [ ]:
dus = list(oespruce.OEMakeDesignUnits(spruced, metadata, opts, "TYR:161: :A"))

In [ ]:
dus = list(oespruce.OEMakeDesignUnits(spruced, metadata, opts, "TYR:161: :A:0: "))

## Try using the BioDesignUnits function

In [ ]:
biodu = oespruce.OEMakeBioDesignUnits(spruced, metadata, opts)

In [ ]:
dus = [du for du in biodu]

In [ ]:
dus[0]

In [ ]:
oespruce.OEFindPockets(dus[0])

In [ ]:
oespruce.OEEnumerateSites(dus[0], oespruce.OEDesignUnitEnumerateSitesOptions(), "TYR:161: :B:0: ")

## try to get site residue from siyang's function

In [ ]:
def find_oe_res_id(protein):
    #### Get the residues from the molecule
    residues = oechem.OEGetResidues(protein)
    # Getting the residue ids in the protein file
    oe_id = dict()
    count = 1
    for residue in residues:
        #oe_thing[(residue.GetName(),residue.GetResidueNumber(),residue.GetExtChainID())] = count
        # Since did not get the chain ID before, would just go with A now
        oe_id[count] = (residue.GetName(),residue.GetResidueNumber(),residue.GetChainID())
        count += 1
    return oe_id

In [ ]:
find_oe_res_id(spruced)

# conclusion

Yeah I have no idea why this isn't working. I think this is a bit of a dead end.

# Double Check to see if apo prepping works for mpro

In [ ]:
from asapdiscovery.data.metadata.resources import master_structures

In [ ]:
mpro_path = master_structures["SARS-CoV-2-Mpro"]

In [ ]:
mpro = load_openeye_pdb(mpro_path)

In [ ]:
find_oe_res_id(mpro)

In [ ]:
from asapdiscovery.modeling.modeling import split_openeye_mol

In [ ]:
apo = split_openeye_mol(mpro, 'protein')['prot']

In [ ]:
success, spruce_error_message, spruced = spruce_protein(apo)

In [ ]:
success

In [ ]:
find_oe_res_id(spruced)

In [ ]:
dus = list(oespruce.OEMakeDesignUnits(spruced, metadata, opts, "HIS:163: :A"))

In [ ]:
dus[0].HasLigand()

In [ ]:
dus[0].HasReceptor()

In [ ]:
dus[0].HasProtein()

In [ ]:
du = dus[0]

In [ ]:
from asapdiscovery.data.backend.openeye import oedocking
oedocking.OEMakeReceptor(du)

In [ ]:
du.HasReceptor()

# it does work for the apo structure of mpro

In [ ]:
from asapdiscovery.modeling.modeling import du_to_complex

In [ ]:
prepped_target = du_to_complex(dus[0])

In [ ]:
save_openeye_pdb(prepped_target, f"SARS-CoV-2-Mpro_apo.pdb")

## test using this du to dock

In [ ]:
lig = split_openeye_mol(mpro, 'protein')['lig']

In [ ]:
lig.NumAtoms()

In [ ]:
lig = oechem.OEMol(lig)

In [ ]:
from asapdiscovery.data.backend.openeye import oedocking

In [ ]:
poser = oedocking.OEPosit()
pose_res = oedocking.OEPositResults()
retcodes = [poser.AddReceptor(du) for du in dus]
retcode = poser.Dock(pose_res, lig, 1)

In [ ]:
poser.AddReceptor(dus[0])

In [ ]:
oedocking.OEDockingReturnCodeGetName(retcode)

# This works!

# Conclusion

So it definitely *can* work. It just doesn't in our case with the zika fragments for some reason. So let's use the method of making the DU directly, and then re-docking into that DU.